In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from rnn_dataset import get_loader

In [2]:
class RNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNNCell, self).__init__()
        self.hidden_size = hidden_size
        self.whh = nn.Linear(hidden_size, hidden_size)
        self.wxh = nn.Linear(input_size, hidden_size)
    
    def forward(self, x, hidden_state): # x (batch_size, input_size)
        return torch.sigmoid(self.whh(hidden_state) + self.wxh(x))

In [3]:
class SentimentClassifier(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, output_size=2):
        super(SentimentClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.rnn1 = RNNCell(input_size, hidden_size)
        self.rnn2 = RNNCell(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
#         self.whh = nn.Linear(hidden_size, hidden_size)
#         self.wxh = nn.Linear(input_size, hidden_size)
        
    
    def init_hidden_state(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)
    
    def forward(self, x, hidden_state): # x (seq_len, batch_size, input_size)
        h1 = hidden_state
        h2 = hidden_state

        for i in range(x.shape[0]):
            h1 = self.rnn1(x[i], h1)
            h2 = self.rnn2(h1, h2)
        
        output = F.softmax(self.fc(h2), dim=1)
        return output, hidden_state
    
    def fit(self, dataset, batch_size, epochs, lr=0.001):
        optimizer = torch.optim.SGD(self.parameters(), lr=lr)
        criterion = torch.nn.CrossEntropyLoss() # ignore_index=pad_idx?
        
        for epoch in range(epochs):
            total_loss = 0
            for idx, (texts, sentiments) in enumerate(dataset):
                hidden_state = self.init_hidden_state(batch_size=batch_size)
                
                # forward
                for i in range(texts.shape[0]):
                    output, hidden_state = self.forward(texts[i], hidden_state)
                
                loss = criterion(output, sentiments)
                
                # backward
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1)
                
                # gradient descent or Adam step
                optimizer.step()
                
                total_loss += loss
            
            if (epoch % 10 == 0): print(f"epoch [{epoch+1} / {epochs}] | total loss: {total_loss}")

In [46]:
BATCH_SIZE=1
HIDDEN_SIZE=256
OUTPUT_SIZE=2
LR=0.5
NUM_EPOCHS=200

In [53]:
# LOADING THE DATA
dataloader,dataset = get_loader("../data/", "small_sentiments.csv", batch_size=BATCH_SIZE)

In [48]:
classifier = SentimentClassifier(input_size=dataset.vocab_size, hidden_size=HIDDEN_SIZE,
                                output_size=OUTPUT_SIZE)

In [54]:
# TRAINING
classifier.fit(dataloader, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, lr=LR)

epoch [1 / 200] | total loss: 1.3130816221237183
epoch [11 / 200] | total loss: 1.313055157661438
epoch [21 / 200] | total loss: 1.3130193948745728
epoch [31 / 200] | total loss: 1.3129692077636719
epoch [41 / 200] | total loss: 1.3128929138183594
epoch [51 / 200] | total loss: 1.3127641677856445
epoch [61 / 200] | total loss: 1.3125016689300537
epoch [71 / 200] | total loss: 1.3116923570632935
epoch [81 / 200] | total loss: 1.2842903137207031
epoch [91 / 200] | total loss: 0.31467482447624207
epoch [101 / 200] | total loss: 0.31396782398223877
epoch [111 / 200] | total loss: 0.3137337267398834
epoch [121 / 200] | total loss: 0.3136162757873535
epoch [131 / 200] | total loss: 0.31354570388793945
epoch [141 / 200] | total loss: 0.31349849700927734
epoch [151 / 200] | total loss: 0.3134647011756897
epoch [161 / 200] | total loss: 0.31343942880630493
epoch [171 / 200] | total loss: 0.3134196400642395
epoch [181 / 200] | total loss: 0.3134038746356964
epoch [191 / 200] | total loss: 0.3133

In [55]:
def get_sentiment(classifier, vocab, sentence):
    vocab_size = len(vocab)
    
    def one_hot_tensor(idx):
        tensor = [0] * vocab_size
        tensor[idx] = 1
        return tensor
    
    encoded_text = []
    encoded_text.append([one_hot_tensor(vocab.stoi["<SOS>"])])
    encoded_text += [[one_hot_tensor(encoded_token)]
                     for encoded_token in vocab.encode(sentence)]
    encoded_text.append([one_hot_tensor(vocab.stoi["<EOS>"])])

    encoded_text = torch.tensor(encoded_text).float()
    print(encoded_text.shape)
    h0 = classifier.init_hidden_state(batch_size=1)

    for i in range(encoded_text.shape[0]):
        output, h0 = classifier(encoded_text[i], h0)
    return output

In [56]:
classifier.eval()
get_sentiment(classifier, dataset.vocab, "I love you")

torch.Size([5, 1, 7])


tensor([[9.9978e-01, 2.2009e-04]], grad_fn=<SoftmaxBackward>)